In [9]:
import cv2
import numpy as np

# Carica l'immagine RGB
rgb_image = cv2.imread('output/camera1.png')

# Assicurati che l'immagine sia in formato RGB (a volte OpenCV carica come BGR)
rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB)

# Calcola la profondità normalizzata
normalized_depth = (rgb_image[:, :, 0] + rgb_image[:, :, 1] * 256 + rgb_image[:, :, 2] * 256**2) / (256**3 - 1)

# Converte la profondità in metri
depth_in_meters = 1000 * normalized_depth
print(depth_in_meters)

depth_grayscale = cv2.normalize(depth_in_meters, None, 0, 255, cv2.NORM_MINMAX)
depth_grayscale = depth_grayscale.astype(np.uint8) 
# Visualizza l'immagine della profondità
cv2.imshow('Depth in Meters', depth_grayscale)  # Visualizzazione in formato intero per chiarezza
cv2.waitKey(0)
cv2.destroyAllWindows()


[[27.91404891 28.00399232 28.09447218 ... 37.64808402 37.57071719
  37.49364838]
 [27.90433335 27.99415755 28.08463741 ... 37.65648828 37.57912145
  37.50199303]
 [27.89461779 27.98438239 28.07480264 ... 37.66489253 37.5874661
  37.51033768]
 ...
 [ 8.98206288  8.98212248  8.98218208 ...  8.98283774  8.98283774
   8.98283774]
 [ 8.96775776  8.66645626  8.66627745 ...  8.96817499  8.96817499
   8.96817499]
 [ 8.6425548   8.64750198  8.6528664  ...  8.95351225  8.95351225
   8.95351225]]


In [11]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Funzione per creare l'immagine 2D dall'alto
def create_top_down_view(depth_image, rgb_image, camera_position, focal_length, sensor_size, scale=1.0):
    """
    Crea un'immagine 2D dall'alto a partire da una mappa di profondità e un'immagine RGB.
    
    :param depth_image: Immagine in scala di grigi della profondità (H, W).
    :param rgb_image: Immagine RGB (H, W, 3).
    :param camera_position: Posizione della telecamera come tuple (x, y, z).
    :param focal_length: Lunghezza focale della telecamera.
    :param sensor_size: Dimensioni del sensore della telecamera (w, h).
    :param scale: Scala per adattare la profondità all'immagine finale.
    :return: Immagine 2D dell'area vista dall'alto.
    """
    height, width = np.shape(depth_image)
    camera_x, camera_y, camera_z = camera_position
    
    # Creazione di un'immagine vuota per la vista dall'alto
    top_down_image = np.zeros((height, width, 3), dtype=np.uint8)
    
    # Calcolo dei parametri di proiezione
    pixel_size_x = sensor_size[0] / width
    pixel_size_y = sensor_size[1] / height
    
    # Iterazione su ciascun pixel per costruire la vista dall'alto
    for y in range(height):
        for x in range(width):
            depth = depth_image[y, x] * scale  # Profondità scalata

            # Calcolo delle coordinate 3D nel mondo
            world_x = (x - width / 2) * pixel_size_x * focal_length / depth
            world_y = (y - height / 2) * pixel_size_y * focal_length / depth
            world_z = depth

            # Traslazione della posizione della telecamera
            world_x += camera_x
            world_y += camera_y
            world_z += camera_z

            # Proiezione su un piano 2D (assumendo la vista dall'alto, con l'asse z come altezza)
            if 0 <= int(world_x) < width and 0 <= int(world_y) < height:
                top_down_image[int(world_y), int(world_x)] = rgb_image[y, x]
    
    return top_down_image

# Caricamento delle immagini
depth_image = depth_grayscale
rgb_image = cv2.imread('output/camera.png')

# Parametri della telecamera
camera_position = (0, 0, 1000)  # Esempio: la telecamera è posizionata a 1000 unità sopra l'origine
focal_length = 800  # Lunghezza focale in pixel
sensor_size = (0.036, 0.024)  # Dimensioni del sensore in metri (esempio per una fotocamera standard)

# Creazione della vista dall'alto
top_down_view = create_top_down_view(depth_image, rgb_image, camera_position, focal_length, sensor_size)

# Visualizzazione dell'immagine risultante
plt.imshow(top_down_view)
plt.title('Vista dallalto')
plt.show()


c:\Users\danie\anaconda3\envs\carla-env\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars
c:\Users\danie\anaconda3\envs\carla-env\lib\site-packages\ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in double_scalars


OverflowError: cannot convert float infinity to integer